In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, f1_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from pandas import MultiIndex, Int64Index
# models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
import xgboost as xgb

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import optuna
import pandas as pd
from sklearn import ensemble
from sklearn import datasets
from sklearn import model_selection

from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
from sklearn.model_selection import train_test_split
import bokeh



/var/folders/32/zv3nm8_12xb9ytd7vk7nnr6w0000gn/T/ipykernel_61970/3489285944.py:8: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
#!pip3 install xgboost
#!pip3 install optuna
#!pip install autogluon --upgrade
# !pip install optuna
# !pip install scipy
# !pip install autogluon
# !pip install xgboost
# !python3.9 -m pip install --upgrade pip
#!pip install bokeh==2.0.1
#!pip install jupyter --upgrade


In [3]:
#!conda install mxnet
#import mxnet

In [4]:
df_train = pd.read_csv('../data/train_encoded_1.csv')
df_test = pd.read_csv('../data/test_encoded_1.csv')

In [5]:
np.random.seed(0)
train, valid = train_test_split(df_train, test_size=0.2)

In [6]:
combined = df_train.drop(['price', 'size_sqft'], axis=1)
X_train = train.drop(['price', 'size_sqft'], axis=1)

X_valid = valid.drop(['price', 'size_sqft'], axis=1)
y_valid = valid['price_per_sqft']

df_test_size_sqft = df_test['size_sqft'].copy()
df_test= df_test.drop(['size_sqft'], axis=1)

metric_reg = 'mean_absolute_error'
label_reg = 'price_per_sqft'
train_data_reg = TabularDataset(X_train) 
valid_data_reg = TabularDataset(X_valid)
test_data_reg = TabularDataset(df_test)
combined = TabularDataset(combined)



Tabular Predictor

In [ ]:
predictor = TabularPredictor(label = label_reg, eval_metric = metric_reg, path = 'Autogluon_20k_featureset')

gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
    'num_leaves': ag.space.Int(lower=26, upper=66, default=36),  # number of leaves in trees (integer hyperparameter)
}

hyperparameters = {
 'NN_TORCH': {},
 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}],
 'CAT': {},
 #'NN_MXNET':{},
 'XT':{}
}

time_limit = 30*60  # train various models for ~5 min
num_trials = 3  # try at most 5 different hyperparameter configurations for each type of model
search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
    'num_trials': num_trials,
    'scheduler' : 'local',
    'searcher': search_strategy,
}



#excluded_model_types = ['NN_TORCH', 'FASTAI', 'KNN', 'CAT', 'RF']
excluded_model_types = ['FASTAI', 'KNN', 'RF', 'XGB']
predictor_reg = predictor.fit(combined, excluded_model_types = excluded_model_types
,hyperparameters=hyperparameters
#,tuning_data=valid_data_reg
,hyperparameter_tune_kwargs=hyperparameter_tune_kwargs, presets='best_quality'

)


Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "Autogluon_20k_featureset/"
AutoGluon Version:  0.5.2
Python Version:     3.9.7
Operating System:   Darwin
Train Data Rows:    20147
Train Data Columns: 31
Label Column: price_per_sqft
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4486.890450010187, 447.8293983244478, 1719.76428, 982.28364)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    6123.77 MB
	Train Data (Orig

  0%|          | 0/3 [00:00<?, ?it/s]

[1000]	valid_set's l1: 145.632
[2000]	valid_set's l1: 142.047
[3000]	valid_set's l1: 140.938
[4000]	valid_set's l1: 140.29
[5000]	valid_set's l1: 140.097
[6000]	valid_set's l1: 139.92
[7000]	valid_set's l1: 139.972
[1000]	valid_set's l1: 139.495
[1000]	valid_set's l1: 146.828
[2000]	valid_set's l1: 143.577
[3000]	valid_set's l1: 142.29
[4000]	valid_set's l1: 141.629
[5000]	valid_set's l1: 141.449
[6000]	valid_set's l1: 141.407
[7000]	valid_set's l1: 141.43


Fitted model: LightGBMXT_BAG_L1/T1 ...
	-139.8374	 = Validation score   (-mean_absolute_error)
	7.93s	 = Training   runtime
	0.34s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1/T2 ...
	-139.1468	 = Validation score   (-mean_absolute_error)
	3.63s	 = Training   runtime
	0.09s	 = Validation runtime
Fitted model: LightGBMXT_BAG_L1/T3 ...
	-141.3145	 = Validation score   (-mean_absolute_error)
	13.4s	 = Training   runtime
	0.55s	 = Validation runtime
Hyperparameter tuning model: CatBoost_BAG_L1 ...


  0%|          | 0/3 [00:00<?, ?it/s]

...

	-142.2841	 = Validation score   (-mean_absolute_error)
	68.35s	 = Training   runtime
	3.16s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1/T2 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 138.228
[2000]	valid_set's l1: 137.007
[3000]	valid_set's l1: 137.016
[1000]	valid_set's l1: 143.219
[1000]	valid_set's l1: 139.606
[2000]	valid_set's l1: 138.823
[1000]	valid_set's l1: 141.252
[2000]	valid_set's l1: 140.5
[1000]	valid_set's l1: 140.4


	-140.4737	 = Validation score   (-mean_absolute_error)
	32.21s	 = Training   runtime
	0.84s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1/T3 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 145.935
[2000]	valid_set's l1: 143.419
[3000]	valid_set's l1: 142.617
[4000]	valid_set's l1: 142.218
[5000]	valid_set's l1: 142.238
[6000]	valid_set's l1: 142.352
[1000]	valid_set's l1: 150.89
[2000]	valid_set's l1: 147.954
[3000]	valid_set's l1: 147.727
[1000]	valid_set's l1: 148.211
[2000]	valid_set's l1: 145.611
[3000]	valid_set's l1: 145.164
[4000]	valid_set's l1: 144.682
[5000]	valid_set's l1: 144.812
[1000]	valid_set's l1: 150.253
[2000]	valid_set's l1: 146.852
[3000]	valid_set's l1: 145.301
[4000]	valid_set's l1: 144.615
[5000]	valid_set's l1: 144.455
[6000]	valid_set's l1: 144.25
[7000]	valid_set's l1: 144.255
[8000]	valid_set's l1: 144.25
[1000]	valid_set's l1: 149.982
[2000]	valid_set's l1: 147.42
[3000]	valid_set's l1: 146.537
[4000]	valid_set's l1: 146.069
[5000]	valid_set's l1: 146.03
[6000]	valid_set's l1: 146.014
[7000]	valid_set's l1: 145.788
[8000]	valid_set's l1: 145.83
[9000]	valid_set's l1: 145.934
[1000]	valid_set's l1: 147.454
[2000]	valid_s

	-143.8167	 = Validation score   (-mean_absolute_error)
	94.63s	 = Training   runtime
	3.94s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T1 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-139.1353	 = Validation score   (-mean_absolute_error)
	1381.51s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T2 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-140.3417	 = Validation score   (-mean_absolute_error)
	1230.48s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: CatBoost_BAG_L1/T3 ...
	Fitting 7 child models (S1F2 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


In [25]:
#/Users/nishita/Documents/Semester 1/CS5228/project/notebooks/AutogluonModels/ag-20221027_054028
folderName = "AutogluonModels/ag-20221028_001415/"
predictor = TabularPredictor.load(folderName)
predictor.fit_hyperparameters_

{'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}],
 'CAT': {},
 'XT': {}}

In [32]:
predictor.evaluate(valid_data_reg)

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

{'mean_absolute_error': -145.6304617727714,
 'root_mean_squared_error': -292.88972240525686,
 'mean_squared_error': -85784.38949062843,
 'r2': 0.9066739116978073,
 'pearsonr': 0.9522197563763414,
 'median_absolute_error': -74.19214787232931}

In [27]:
#With target encoding and less one hot encoding.
predictor = TabularPredictor.load(folderName)
results = predictor.fit_summary(show_plot=True)

def writePredictionsToFile(modelName, filename, folder = folderName, refit = False):
    if(refit == True):
        modelName = modelName +  '_FULL'    
    y_pred = predictor.predict(test_data_reg, model=modelName)
    yTest = y_pred * df_test_size_sqft
    yTest = pd.DataFrame(yTest, columns=['Predicted'], index=df_test.index)
    yTest.to_csv(f'{folder}/predictions/{filename}.csv')


*** Summary of fit() ***
Estimated performance of each model:
                   model   score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3 -142.068417       8.135790  4147.233197                0.000648           0.317045            3       True         14
1    WeightedEnsemble_L2 -142.819399       1.044720  3722.726048                0.000689           0.312974            2       True          7
2     CatBoost_BAG_L1/T3 -143.111538       0.090473  1497.658844                0.090473        1497.658844            1       True          6
3     CatBoost_BAG_L1/T1 -144.831912       0.068832  1319.087439                0.068832        1319.087439            1       True          4
4     CatBoost_BAG_L2/T3 -145.295000       6.332602  4048.957216                0.042637         191.765848            2       True         13
5   LightGBMXT_BAG_L2/T2 -145.501135       8.025588  3922.734464                

In [28]:
predictor.leaderboard(valid_data_reg)
predictor.evaluate(valid_data_reg)

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

                   model  score_test   score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L3 -145.630462 -142.068417       19.243859       8.135790  4147.233197                 0.004866                0.000648           0.317045            3       True         14
1     CatBoost_BAG_L2/T3 -146.081631 -145.295000       14.932324       6.332602  4048.957216                 0.117428                0.042637         191.765848            2       True         13
2    WeightedEnsemble_L2 -146.426780 -142.819399        2.739450       1.044720  3722.726048                 0.006107                0.000689           0.312974            2       True          7
3     CatBoost_BAG_L1/T1 -146.484567 -144.831912        0.185685       0.068832  1319.087439                 0.185685                0.068832        1319.087439            1       True          4
4     CatBoost_BAG_L

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items i

{'mean_absolute_error': -145.6304617727714,
 'root_mean_squared_error': -292.88972240525686,
 'mean_squared_error': -85784.38949062843,
 'r2': 0.9066739116978073,
 'pearsonr': 0.9522197563763414,
 'median_absolute_error': -74.19214787232931}

In [30]:
writePredictionsToFile(modelName = 'WeightedEnsemble_L2', filename = 'WeightedEnsemble_L2')


/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [31]:
writePredictionsToFile(modelName = 'CatBoost_BAG_L2/T3', filename = 'CatBoost')
writePredictionsToFile(modelName = 'LightGBMXT_BAG_L2/T2', filename = 'LightGBMXT')

/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/usr/local/lib/python3.9/site-packages/catboost/core.py:1411: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


In [ ]:
#predictor.refit_full()